In [2]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")
# Note: pipeline is not directly compatible with LLMChain → You need to wrap it using HuggingFacePipeline.
# LangChain expects structured output, while pipeline returns a list of dictionaries (which must be handled properly).
llm = HuggingFacePipeline(pipeline=hf_pipeline) #use this to create a LLM chain

Device set to use cpu


In [3]:

from langchain.prompts import PromptTemplate
#define prompt here
prompt_template = PromptTemplate(
    input_variables=['country'],
    template = "Tell me the capital of  {country} ?")

formatted_prompt = prompt_template.format(country='India')

In [4]:
# Firstly, directly using the pipeline here without any chains
response = hf_pipeline(formatted_prompt)
print(response[0]['generated_text'])

chennai


In [5]:
# Chains concept here
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template)
chain.run("India")

/tmp/ipykernel_3788/1650880420.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt = prompt_template)
/tmp/ipykernel_3788/1650880420.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("India")


'chennai'

### Combining multiple chains here

In [6]:
capital_template=PromptTemplate(
    input_variables=['country'],
    template="Please tell me the capital of the {country}"
)
capital_chain = LLMChain(llm=llm, prompt=capital_template)
#I create two chains, above abnd below
famous_template = PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}"
)
famous_chain = LLMChain(llm = llm, prompt=famous_template)

In [7]:
#get outpt from the first chain and pass it to the another chain
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("Canada")

'Toronto Museum of Art'

### SequentialChain

In [8]:
#getting answers of all the chains
capital_template=PromptTemplate(
    input_variables=['country'],
    template="Please tell me the capital of the {country}"
)
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

famous_template = PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}"
)
famous_chain = LLMChain(llm = llm, prompt=famous_template, output_key="places")

In [9]:
from langchain.chains import SequentialChain

chain=SequentialChain(
    chains=[capital_chain, famous_chain],
    input_variables=["country"],
    output_variables=["capital","places"]
    )
chain({'country: "India'})

/tmp/ipykernel_3788/1546279761.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'country: "India'})


{'country': {'country: "India'},
 'capital': 'calcutta',
 'places': 'The National Museum of India'}

### Chatmodels

In [10]:
from langchain_community.llms import HuggingFacePipeline #for hugging face models
from langchain.schema import HumanMessage, SystemMessage, AIMessage
Chatllm = HuggingFacePipeline(pipeline=hf_pipeline)

system_message = SystemMessage(content="You are a comedian AI assistant")
human_message = HumanMessage(content="Please make some jokes on AI")

# Join the content of the messages into one string
final_input = "\n".join([system_message.content, human_message.content])

Chatllm(final_input)

/tmp/ipykernel_3788/2086509714.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  Chatllm(final_input)


'AI assistants are a joke'

### Prompt Template + LLM + Outout Parsers

In [11]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [12]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")

In [13]:
# created a chat prompt
template="You area  helpful assistant. When the use of given any input, you should generate five words sysnsynonym  in a comma seperated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template), 
    ("human", human_template)
])

chain = chatprompt|Chatllm|Commaseperatedoutput()

In [14]:
chain.invoke({"text":"good"})

['good human']